# 05 - Regression

###### Here, to build a predictive model able to anticipate possible changes and additions in the dataset (in this case, the finished operations historical) a regression model was developed. The regression model seeks to identify patterns in the current dataset and predict the behaviour of additional informations (related to finished operations).

##### Considered Regression Models:

###### - Linear Regression;
###### - Ridge;
###### - Lasso;
###### - ElastiicNet;
###### - Decision Tree;
###### - Random Forest;
###### - Gradient Boosting;
###### - XGBoost;
###### - Support Vector Regressor (SVR)
    
----

#### Loading Packages and Database
----

In [1]:
import plotly.io as pio
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
# import fast_dummies
from sklearn.preprocessing import OneHotEncoder
from scipy import stats
import os
import sys
from pathlib import Path

In [2]:
# Adicionar o diretório 'modules_py' ao sys.path
modules_path = Path("modules_py").resolve()
sys.path.append(str(modules_path))

from data_cleaning import DataCleaning_MyData, DataCleaning_GroupData, DataCleaning_OffCod, DataCleaning_frequents, DataCleaning_dummies
from aplicaRegressao import aplicar_modelos_regressao

try:
    # Executar a função principal de limpeza de dados
    my_data = DataCleaning_MyData()
    clean_media = DataCleaning_GroupData()
    OffCod_Agru = DataCleaning_OffCod()
    mais_frequentes = DataCleaning_frequents()
    dummies = DataCleaning_dummies()
    
    print("Dados processados com sucesso!")
    display(OffCod_Agru)
    

# Mostrar o erro caso não execute
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Dados processados com sucesso!


,OFFCod,Andrea_DEPARTAMENTO.Descricao,TipodeProduto,Produto.Nome,Poder Linear,Poder Z,SomaDeHorasApontadasUnitario
0,66,CONFORMACAO,Caldeira,CRCHRSG,NaN,0.0,7066.0
1,66,MONTAGEM,Caldeira,CRCHRSG,NaN,0.0,6760.0
2,66,SOLDA,Caldeira,CRCHRSG,NaN,0.0,4220.0
3,66,TRACAGEM,Caldeira,CRCHRSG,NaN,0.0,2400.0
4,66,USINAGEM,Caldeira,CRCHRSG,NaN,0.0,7174.0
...,...,...,...,...,...,...,...
397,LWarcel,CONFORMACAO,Caldeira,CRQ,NaN,0.0,24113.0
398,LWarcel,MONTAGEM,Caldeira,CRQ,NaN,0.0,49286.0
399,LWarcel,SOLDA,Caldeira,CRQ,NaN,0.0,77425.0
400,LWarcel,TRACAGEM,Caldeira,CRQ,NaN,0.0,5617.0


In [3]:
# Usando o filtro do OffCod
OffCod = OffCod_Agru[OffCod_Agru["SomaDeHorasApontadasUnitario"].notna()]
OffCod = OffCod[OffCod["SomaDeHorasApontadasUnitario"] != 0]

# Removendo colunas com dados faltantes
OffCod = OffCod.drop(['Poder Linear', 'Poder Z'], axis= 1)

# Lista de departamentos
departamentos_list = ['CONFORMACAO', 'MONTAGEM', 'SOLDA', 'TRACAGEM', 'USINAGEM']

# Dicionário para armazenar os DataFrames
Departamento = {}

# Criando os DataFrames com um loop
for d in departamentos_list:
    Departamento[d] = OffCod[
        (OffCod['TipodeProduto'] == 'Caldeira') &
        (OffCod['Andrea_DEPARTAMENTO.Descricao'] == d)
    ]

    # Exibir o número de registros para cada departamento
    print(f"Departamento: {d}, Registros: {len(Departamento[d])}")

# Criando uma lista de dataframes
dfs = [
    Departamento['CONFORMACAO'],
    Departamento['MONTAGEM'],
    Departamento['SOLDA'],
    Departamento['TRACAGEM'],
    Departamento['USINAGEM']
    ]

display(Departamento['CONFORMACAO'])
OffCod.dtypes

Departamento: CONFORMACAO, Registros: 46
Departamento: MONTAGEM, Registros: 56
Departamento: SOLDA, Registros: 55
Departamento: TRACAGEM, Registros: 53
Departamento: USINAGEM, Registros: 53


,OFFCod,Andrea_DEPARTAMENTO.Descricao,TipodeProduto,Produto.Nome,SomaDeHorasApontadasUnitario
0,66,CONFORMACAO,Caldeira,CRCHRSG,7066.00
5,71,CONFORMACAO,Caldeira,CQOG,13513.50
26,115,CONFORMACAO,Caldeira,CRCHRSG,1823.00
31,117,CONFORMACAO,Caldeira,CRQ,604.00
44,132,CONFORMACAO,Caldeira,CRQ,1353.00
49,133,CONFORMACAO,Caldeira,CQOG,6899.00
54,134,CONFORMACAO,Caldeira,CC,191.00
59,135,CONFORMACAO,Caldeira,CRCHRSG,7066.00
72,138,CONFORMACAO,Caldeira,CC,328.00
77,139,CONFORMACAO,Caldeira,CND,2715.00


OFFCod                           category
Andrea_DEPARTAMENTO.Descricao    category
TipodeProduto                    category
Produto.Nome                     category
SomaDeHorasApontadasUnitario      float64
dtype: object

In [4]:
resultados = aplicar_modelos_regressao(dfs, target_column='SomaDeHorasApontadasUnitario')

# Exportar resultados para Excel
resultados.to_excel('resultados_regressao.xlsx', index=False)

# Exibir os resultados
print(resultados)

Processando dataset 1...


c:\Users\Pedro Amaral Pereira\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 951294.3622463846, tolerance: 268538.247625
  model = cd_fast.sparse_enet_coordinate_descent(


Processando dataset 2...


c:\Users\Pedro Amaral Pereira\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4373827.11258011, tolerance: 1838110.8994687123
  model = cd_fast.sparse_enet_coordinate_descent(
c:\Users\Pedro Amaral Pereira\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3587413.3050449225, tolerance: 1201043.0851079084
  model = cd_fast.sparse_enet_coordinate_descent(
c:\Users\Pedro Amaral Pereira\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3670344.724287448, tolerance: 1351421.928857494
 

Processando dataset 3...
Processando dataset 4...
Processando dataset 5...
      Dataset                Modelo        R2           MAE          RMSE  \
0   Dataset 1      Regressao Linear -1.843225   6262.755662   7599.443199   
1   Dataset 1                 Ridge -1.725686   6414.460427   7440.705377   
2   Dataset 1                 Lasso  0.248134   3465.292156   3907.926148   
3   Dataset 1            ElasticNet -1.242114   6384.381621   6748.464987   
4   Dataset 1     Arvore de Decisao -0.181168   3414.821429   4898.150090   
5   Dataset 1         Random Forest  0.284639   3054.484107   3811.877499   
6   Dataset 1     Gradient Boosting  0.329082   3202.627713   3691.567320   
7   Dataset 1               XGBoost  0.318355   3216.668422   3720.962282   
8   Dataset 1   SVR (Linear Kernel) -0.141930   4323.589712   4816.105326   
9   Dataset 1      SVR (RBF Kernel) -0.141930   4323.589712   4816.105326   
10  Dataset 1  SVR (Sigmoid Kernel) -0.142816   4324.834948   4817.972041   
1